# Define Audit Process

In [1]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

ERROR: Connection with the name 'Transcend' already exists

In [2]:
%connect name=Transcend

Password ············


Success: 'Transcend' connection established


In [35]:
call adlste_coa.sp_meta_objects_refresh('none');

Success: 0 rows affected

In [38]:
Select * from adlste_coa.coa_meta_objects
where ObjFunction  = 'audit' 
order by ObjType, ProcessName,  ObjType desc;

DatabaseName,TableName,TableKind,ObjType,ObjFunction,ProcessName
ADLSTE_COA,coa_Audit_Issues,V,coa,Audit,Issues
ADLSTE_COA,coa_Audit_Status,V,coa,Audit,Status
ADLSTE_COA,coat_Audit_Issues,T,coat,Audit,Issues
ADLSTE_COA,coat_Audit_Status,T,coat,Audit,Status
ADLSTE_COA,sp_audit_Account_Name,P,sp,audit,Account_Name
ADLSTE_COA,sp_audit_Site_ID,P,sp,audit,Site_ID
ADLSTE_COA,sp_audit_status_update,P,sp,audit,status_update


<br/><br/><br/>
## Audit_Run (sub-stored_procedure)
### for tracking audit columns per row while minimizing storage
- database:    adlste_coa
- table:       coat_audit_run
- view:        coa_audit_run
- stored proc: sp_audit_run

In [48]:
drop table adlste_coa.coat_audit_run

Success: 21 rows affected

In [78]:
Create multiset table adlste_coa.coat_audit_run
(
 Run_ID     INTEGER 
,Site_ID    VARCHAR(128)
,Run_Type   VARCHAR(128)
,Run_Status VARCHAR(128)
,Run_User   VARCHAR(128) Default USER
,Run_TS     Timestamp(0) Default Current_Timestamp(0)
) 
primary index (Site_ID);

Success: 0 rows affected

In [100]:
replace PROCEDURE adlste_coa.sp_audit_log_run (in siteid varchar(128)
                                              ,in runtype varchar(128)
                                              ,in runstatus varchar(128)
                                              ,inout runid integer)
SQL SECURITY CREATOR
BEGIN
DECLARE runidout INTEGER;

    Insert into adlste_coa.coat_audit_run 
    (Run_ID, Site_ID, Run_Type, Run_Status)
        Select 
        coalesce(max(Run_ID+1),1) as Run_ID
        ,:siteid as Site_ID  
        ,:runtype as  Run_Type
        ,:runstatus as Run_Status
        from adlste_coa.coat_audit_run
        where Site_ID = :siteid ;
     
     SELECT Max(Run_ID) as Run_ID INTO runidout 
     FROM adlste_coa.coat_audit_run WHERE Site_ID = :siteid ;
     SET runid = runidout;

END;

Success: 0 rows affected

In [101]:
call adlste_coa.sp_audit_log_run('Test','new insert','executing',0)

runid
4


In [98]:
replace view adlste_coa.coa_audit_run 
as locking row for access
Select * from adlste_coa.coat_audit_run

Success: 0 rows affected

In [102]:
select * from adlste_coa.coa_audit_run
where site_id = 'test'

Run_ID,Site_ID,Run_Type,Run_Status,Run_User,Run_TS
1,Test,new insert,executing,SH186014,2020-04-28 20:37:13
2,Test,new insert,executing,SH186014,2020-04-28 20:37:48
3,Test,new insert,executing,SH186014,2020-04-28 20:37:52
4,Test,new insert,executing,SH186014,2020-04-28 20:43:18
